In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
from sqlalchemy import create_engine

%matplotlib inline
warnings.filterwarnings(action='ignore')

In [2]:
# Connection information
user = 'dsbc_student'
pw = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'weatherinszeged'

In [3]:
# Establish and terminate connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, pw, host, port, db))

weather = pd.read_sql_query('select * from weatherinszeged', con=engine)

engine.dispose()

In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
date                   96453 non-null datetime64[ns, UTC]
summary                96453 non-null object
preciptype             96453 non-null object
temperature            96453 non-null float64
apparenttemperature    96453 non-null float64
humidity               96453 non-null float64
windspeed              96453 non-null float64
windbearing            96453 non-null float64
visibility             96453 non-null float64
loudcover              96453 non-null float64
pressure               96453 non-null float64
dailysummary           96453 non-null object
dtypes: datetime64[ns, UTC](1), float64(8), object(3)
memory usage: 8.8+ MB


In [6]:
weather.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000
mean,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956
std,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000
50%,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000
75%,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


In [7]:
weather.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [10]:
# Independent and dependent variables
X = weather[['humidity', 'windspeed']]
Y = weather['target'] = (weather['apparenttemperature'] - weather['temperature'])

# Add a constant
X = sm.add_constant(X)

# Fit OLS using statsmodels
results = sm.OLS(Y, X).fit()

# Print summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        20:23:15   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.0

Are the estimated coefficients statistically significant? 
- The p-values for ```humidity``` and ```windspeed``` are 0 and less than 0.05 so we can conclude the coefficients are statistically significant.

Are the signs of the estimated coefficients in line with your previous expectations?
- Yes, the signs (negative for both) are in line with my previous expectations.

What are the relations between the target and the explanatory variables (interpret the estimated coefficients)? 
- On average, a one point increase in humidity results in 3.0292 decrease in temperature and a one point increase in windspeed results in a 0.1193 decrease in temperature.

In [11]:
# Variable interaction
weather['humid_wdsp_interaction'] = weather['humidity'] * weather['windspeed']

# Update independent variables
X1 = weather[['humid_wdsp_interaction', 'humidity', 'windspeed']]

# Add a constant
X1 = sm.add_constant(X1)

# Fit OLS using statsmodels
results = sm.OLS(Y, X1).fit()

# Print summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 15 Jan 2020   Prob (F-statistic):               0.00
Time:                        21:08:20   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0

Are the coefficients statistically significant? 
- Yes, the coefficients are statistically significant since the p-value is 0.

Did the signs of the estimated coefficients for humidity and windspeed change? 
- Yes, the signs change from negative to positive.

Interpret the estimated coefficients.
- On average, a one point increase amongst a humidity / windspeed interaction results in a temperature decrease of 0.2971. However, a one point increase in humidity results in a temperature increase of 0.1775 and a one point increase in windspeed results in a temperature increase of 0.0905.